You can read about dataset and download it [here](https://www.osha.gov/Establishment-Specific-Injury-and-Illness-Data).

Let's import standard packages:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets
from IPython.display import display

In [2]:
from constants import INSPECTION_DETAILS_FOLDER_NAME, MAPPING_FILES_FOLDER_NAME, ALL, STATE_NAMES, TWO_DIGIT_NAICS
from helpers import get_naics_sector_numbers_by_names
from scrapping_inspection_details import parse_inspection_file, Inspection, get_inspection_details_list

Below is dictionary with file names and corresponding encoding, used during reading of files. File names for most years are commented because we're interested only in three latest years.

In [3]:
FILE_NAMES_ENCODING = {
#     'ITA Data CY 2016.zip': 'cp1252',
#     'ITA Data CY 2017.zip': 'cp1252',
#     'ITA Data CY 2018.zip': 'cp1252',
#     'ITA Data CY 2019.zip': 'utf-8',
    'ITA Data CY 2020.zip': 'utf-8',
    'ITA Data CY 2021 submitted thru 8-29-2022.zip': 'utf-8',
}
OWNERSHIP_MAP = {
    'Not a government entity': 1,
    'State Government entity': 2,
    'Local Government entity': 3,
}

In [4]:
df = pd.concat(list(map(
    lambda name: pd.read_csv(name, encoding=FILE_NAMES_ENCODING[name], low_memory=False),
    FILE_NAMES_ENCODING.keys()
)), ignore_index=True)

Let's delete columns we don't need, delete rows with empty column **year_filing_for**, and reindex dataframe:

In [5]:
df = df.drop(columns=[
    'id', # Unique number for each record
    'street_address',
    'zip_code',
    'no_injuries_illnesses', # Whether the establishment had any OSHA recordable work-related injuries or illnesses during the year
    'total_other_cases',
    'total_skin_disorders',
    'total_poisonings',
    'total_respiratory_conditions',
    'total_hearing_loss',
    'total_other_illnesses',
    'created_timestamp', # The date and time a record was submitted to the ITA
    'change_reason', # The reason why an establishment’s injury and illness summary was changed, if applicable
])

df = df[~df['year_filing_for'].isna()]
df = df[df['annual_average_employees'] < 1000000]
df = df[df['total_hours_worked'] >= 0]
df = df[df['total_dafw_days'] >= 0]
df = df[df['total_djtr_days'] >= 0]
df.index = list(range(len(df)))
df

,company_name,establishment_name,ein,city,state,naics_code,industry_description,annual_average_employees,total_hours_worked,total_deaths,total_dafw_cases,total_djtr_cases,total_dafw_days,total_djtr_days,total_injuries,establishment_id,establishment_type,size,year_filing_for
0,"CCL Tube (Wilkes-Barre), Inc.",CCL Tube,NaN,Wilkes Barre,PA,561910,Labeling services,185.0,386835.0,0.0,0.0,2.0,0.0,49.0,4.0,93030.0,1.0,2.0,2020.0
1,"Del-Co Water Company, Inc.",All Locations,310749383.0,Delaware,OH,221310,Water treatment and distribution,104.0,216320.0,0.0,2.0,0.0,52.0,19.0,2.0,85507.0,1.0,2.0,2020.0
2,DURATECH INDUSTRIES INTERNATIONAL INC,DURATECH INDUSTRIES INTERNATIONAL INC,450283636.0,JAMESTOWN,ND,333111,Bale throwers manufacturing,87.0,206200.0,0.0,1.0,1.0,17.0,12.0,5.0,649852.0,1.0,2.0,2020.0
3,Fred Wahl Marine Construction,Fred Wahl Marine Construction,931029886.0,Reedsport,OR,336611,Naval ship building,109.0,215620.0,0.0,6.0,4.0,451.0,46.0,15.0,541133.0,1.0,2.0,2020.0
4,"J.R. Johnson, LLC","J.R. Johnson, LLC",931153731.0,Portland,OR,236116,"Construction management, multifamily building",142.0,268700.0,0.0,4.0,2.0,38.0,98.0,15.0,407708.0,1.0,2.0,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601260,Ruston Glass & Mirror,Ruston Glass & Mirror,720492530.0,Ruston,LA,238150,"Curtain wall, glass, installation",31.0,66250.0,0.0,0.0,0.0,0.0,0.0,0.0,904557.0,1.0,2.0,2021.0
601261,Coteau des Prairies Health Care System,Coteau des Prairies Hospital,460278210.0,Sisseton,SD,622110,"Hospitals, general medical and surgical",145.0,295784.0,0.0,0.0,0.0,0.0,0.0,8.0,568448.0,1.0,2.0,2021.0
601262,Coteau des Prairies Health Care System,Tekakwitha Living Center,411809912.0,Sisseton,SD,623110,Nursing homes,52.0,82300.0,0.0,0.0,0.0,0.0,0.0,0.0,568449.0,1.0,2.0,2021.0
601263,Heritage of Red Cloud,Heritage of Red Cloud,611815663.0,Red Cloud,NE,623110,Homes for the elderly with nursing care,40.0,80000.0,0.0,0.0,0.0,0.0,0.0,0.0,904560.0,2.0,2.0,2021.0


In [6]:
def my_plot(selected_states, naics, year, ownership_type, employee_number_range, days_of_job_transfer_or_restriction, total_hours_worked_range, days_away_from_work):
    t = df if ALL in selected_states else df[df['state'].isin(selected_states)]
    t = t if ALL in naics else t[t['naics_code'].astype(str).str.startswith(tuple(get_naics_sector_numbers_by_names(naics)))]
    t = t if ALL in year else t[t['year_filing_for'].isin(year)]
    t = t if ALL in ownership_type else t[t['establishment_type'].isin(tuple(map(lambda x: OWNERSHIP_MAP[x], ownership_type)))]
    t = t[t['annual_average_employees'].between(employee_number_range[0], employee_number_range[1])]
    t = t[t['total_hours_worked'].between(total_hours_worked_range[0], total_hours_worked_range[1])]
    t = t[t['total_dafw_days'].between(days_away_from_work[0], days_away_from_work[1])]
    input_df = t[t['total_djtr_days'].between(days_of_job_transfer_or_restriction[0], days_of_job_transfer_or_restriction[1])]

    grouped_df = input_df.groupby(['company_name', 'year_filing_for'])\
    ['annual_average_employees', 'total_hours_worked', 'total_deaths', 'total_dafw_cases', 'total_djtr_cases', 'total_dafw_days', 'total_djtr_days', 'total_injuries']\
    .agg(sum)
    plt.scatter(grouped_df['annual_average_employees'], grouped_df['total_injuries'])
    plt.title("Linear regression between 'Number of Employees' vs 'Number of Injuries' for 2020 and 2021")
    plt.xlabel('Annual Average Number of Employees, millions')
    plt.ylabel('Total number of injuries')

    b, a = np.polyfit(grouped_df['annual_average_employees'], grouped_df['total_injuries'], deg=1)
    xseq = np.linspace(0, max(grouped_df['annual_average_employees']), num=1000)
    plt.plot(xseq, a + b * xseq, color="k", lw=2.5, label='y={:.2f}x+{:.2f}'.format(b,a));
    plt.legend(fontsize=9)

    plt.grid()
    plt.tight_layout()
    plt.show()

In [7]:
t = df.copy(deep=True)
state = widgets.SelectMultiple(
    options=[ALL] + list(STATE_NAMES.keys()),
    value=[ALL],
    rows=10,
    description='State',
)
year = widgets.SelectMultiple(
    options=[ALL] + list(df['year_filing_for'].unique()),
    value=[ALL],
    rows=len(df['year_filing_for'].unique()) + 1,
    description='Year',
)
ownership_type = widgets.SelectMultiple(
    options=[ALL] + list(OWNERSHIP_MAP.keys()),
    value=[ALL],
    rows=len(OWNERSHIP_MAP.keys()) + 1,
    description='Ownership',
)
establishment_type = widgets.SelectMultiple(
    options=[ALL] + list(df['year_filing_for'].unique()),
    value=[ALL],
    rows=len(df['year_filing_for'].unique()) + 1,
    description='Year',
)
naics = widgets.SelectMultiple(
    options=[ALL] + list(set(TWO_DIGIT_NAICS.values())),
    value=[ALL],
    rows=10,
    description='NAICS sector',
    disabled=False
)
annual_average_employees = widgets.IntRangeSlider(
    value=[df['annual_average_employees'].min(), df['annual_average_employees'].max()],
    min=df['annual_average_employees'].min(),
    max=df['annual_average_employees'].max(),
    step=1,
    description='Annual Average Number of Employees:',
    disabled=False,
    continuous_update=False,
    readout=True,
)
total_hours_worked_range = widgets.IntRangeSlider(
    value=[df['total_hours_worked'].min(), df['total_hours_worked'].max()],
    min=df['total_hours_worked'].min(),
    max=df['total_hours_worked'].max(),
    step=1,
    description='Total hours worked by all employees:',
    disabled=False,
    continuous_update=False,
    readout=True,
)
days_away_from_work = widgets.IntRangeSlider(
    value=[df['total_dafw_days'].min(), df['total_dafw_days'].max()],
    min=df['total_dafw_days'].min(),
    max=df['total_dafw_days'].max(),
    step=1,
    description='Days away from work:',
    continuous_update=False,
    readout=True,
)
days_of_job_transfer_or_restriction = widgets.IntRangeSlider(
    value=[df['total_djtr_days'].min(), df['total_djtr_days'].max()],
    min=df['total_djtr_days'].min(),
    max=df['total_djtr_days'].max(),
    step=1,
    description='Days of job transfer or restriction:',
    continuous_update=False,
    readout=True,
)
# total_djtr_days Total number of days of job transfer or restriction
save_as_csv_button = widgets.Button(
    description='Save as CSV file',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    icon='check'
)

def my_filter(selected_states, naics, year, ownership_type, employee_number_range, days_of_job_transfer_or_restriction, total_hours_worked_range, days_away_from_work):
    t = df if ALL in selected_states else df[df['state'].isin(selected_states)]
    t = t if ALL in naics else t[t['naics_code'].astype(str).str.startswith(tuple(get_naics_sector_numbers_by_names(naics)))]
    t = t if ALL in year else t[t['year_filing_for'].isin(year)]
    t = t if ALL in ownership_type else t[t['establishment_type'].isin(tuple(map(lambda x: OWNERSHIP_MAP[x], ownership_type)))]
    t = t[t['annual_average_employees'].between(employee_number_range[0], employee_number_range[1])]
    t = t[t['total_hours_worked'].between(total_hours_worked_range[0], total_hours_worked_range[1])]
    t = t[t['total_dafw_days'].between(days_away_from_work[0], days_away_from_work[1])]
    t = t[t['total_djtr_days'].between(days_of_job_transfer_or_restriction[0], days_of_job_transfer_or_restriction[1])]
    display(t)
    
def save_as_csv(some):
    pd.from_csv(table.outputs[0]['data']['text/plain'])
    table.outputs[0]['data']['text/plain'].to_csv(f'{pd.to_datetime("today").strftime("%Y-%m-%d")}.csv', index=False)

table = widgets.interactive_output(my_filter, {
    'selected_states': state,
    'naics': naics,
    'year': year,
    'ownership_type': ownership_type,
    'employee_number_range': annual_average_employees,
    'total_hours_worked_range': total_hours_worked_range,
    'days_away_from_work': days_away_from_work,
    'days_of_job_transfer_or_restriction': days_of_job_transfer_or_restriction,
})
save_as_csv_button.on_click(save_as_csv, False)
scatter_plot = widgets.interactive_output(my_plot, {
    'selected_states': state,
    'naics': naics,
    'year': year,
    'ownership_type': ownership_type,
    'employee_number_range': annual_average_employees,
    'total_hours_worked_range': total_hours_worked_range,
    'days_away_from_work': days_away_from_work,
    'days_of_job_transfer_or_restriction': days_of_job_transfer_or_restriction,
})

display(state)
display(naics)
display(year)
display(ownership_type)
display(annual_average_employees)
display(total_hours_worked_range)
display(days_away_from_work)
display(days_of_job_transfer_or_restriction)
display(table)
display(save_as_csv_button)
display(scatter_plot)

SelectMultiple(description='State', index=(0,), options=('ALL', 'AL', 'AK', 'AS', 'AZ', 'AR', 'CA', 'CO', 'CT'…

SelectMultiple(description='NAICS sector', index=(0,), options=('ALL', 'Utilities', 'Non-classifiable', 'Whole…

SelectMultiple(description='Year', index=(0,), options=('ALL', 2020.0, 2021.0), rows=3, value=('ALL',))

SelectMultiple(description='Ownership', index=(0,), options=('ALL', 'Not a government entity', 'State Governme…

IntRangeSlider(value=(0, 954471), continuous_update=False, description='Annual Average Number of Employees:', …

...

IntRangeSlider(value=(0, 1442000), continuous_update=False, description='Days away from work:', max=1442000)

IntRangeSlider(value=(0, 1217046), continuous_update=False, description='Days of job transfer or restriction:'…

Output()

Button(description='Save as CSV file', icon='check', style=ButtonStyle())

Output()

In [8]:
df.__dict__.keys()

dict_keys(['_is_copy', '_mgr', '_item_cache', '_attrs', '_flags'])

In [12]:
df['total_dafw_cases'].unique()

array([0.000e+00, 2.000e+00, 1.000e+00, 6.000e+00, 4.000e+00, 9.000e+00,
       1.100e+01, 3.000e+00, 5.000e+00, 8.000e+00, 1.300e+01, 7.000e+00,
       1.000e+01, 4.700e+01, 1.400e+01, 2.100e+01, 1.500e+01, 3.000e+01,
       1.200e+01, 1.600e+01, 1.900e+01, 1.700e+01, 4.000e+01, 1.800e+01,
       4.400e+01, 6.300e+01, 2.500e+01, 2.200e+01, 2.000e+01, 2.460e+02,
       6.000e+01, 2.700e+01, 3.500e+01, 3.400e+01, 8.800e+01, 2.400e+01,
       2.900e+01, 4.100e+01, 4.500e+01, 1.760e+02, 3.200e+01, 4.900e+01,
       2.800e+01, 3.300e+01, 6.100e+01, 3.050e+02, 3.100e+01, 7.600e+01,
       3.900e+01, 5.500e+01, 3.600e+01, 6.200e+01, 6.400e+01, 2.530e+02,
       5.800e+01, 6.600e+01, 2.300e+01, 4.600e+01, 5.600e+01, 7.200e+01,
       1.120e+02, 5.400e+01, 4.800e+01, 1.930e+02, 4.300e+01, 5.700e+01,
       7.000e+01, 6.900e+01, 3.800e+01, 8.000e+01, 4.200e+01, 6.800e+01,
       9.500e+01, 1.130e+02, 1.380e+02, 1.030e+02, 1.310e+02, 3.700e+01,
       1.040e+02, 5.900e+01, 2.600e+01, 1.890e+02, 

In [10]:
grouped_df = df.groupby(['company_name', 'year_filing_for'])\
['annual_average_employees', 'total_hours_worked', 'total_deaths', 'total_dafw_cases', 'total_djtr_cases', 'total_dafw_days', 'total_djtr_days', 'total_injuries']\
.agg(sum)

/tmp/ipykernel_44774/198286160.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_df = df.groupby(['company_name', 'year_filing_for'])\


In [ ]:
[state-filter, year-filter, ownership-filter, naics-filter, days_away_from_work-filter, total_hours_worked-filter, annual_average_employees-filter, days_of_job_transfer_or_restriction-filter, total_deaths-filter, total_dafw_cases-filter, total_djtr_cases-filter, total_injuries-filter]

In [ ]:
["Days away from work", "Total hours worked", "Annual average employees", "Days of job transfer or restriction", "Total Deaths", "Cases with days away from work", "Cases with job transfer or restriction", "Number of injuries"]


In [14]:
QUANTITATIVE_VALUES  = {
    "Days away from work": "total_dafw_days",
    "Total hours worked": "total_hours_worked",
    "Annual average employees": "annual_average_employees",
    "Days of job transfer or restriction": "total_djtr_days",
    "Total Deaths": "total_deaths",
    "Cases with days away from work": "total_dafw_cases",
    "Cases with job transfer or restriction": "total_djtr_cases",
    "Number of injuries": "total_injuries",
}

In [17]:
list(QUANTITATIVE_VALUES.keys())[0]

'Days away from work'